In [41]:
import pandas as pd
import re

In [53]:
#reading in files
df_2012 = pd.read_csv('database_miccai_2012_with_cat.csv')
df_2021 = pd.read_csv('database_miccai_2021_with_cat.csv')

In [54]:
#removing unnecessary index column
df_2012 = df_2012.drop("Unnamed: 0", axis = 1)
df_2021 = df_2021.drop("Unnamed: 0", axis = 1)

In [55]:
#finding the references
def find_reference(year, place):
    reference_list = []
    with open(place, "r", encoding = 'utf-8') as part:
        article = part.read()
        if year == 2012:
            references = [i.start() for i in re.finditer("References\n\n1", article)]
            other_ref = [i.start() for i in re.finditer("References\n\n\[1\]", article)]
            all_ref = references + other_ref
            all_ref.sort()
            
        if year == 2021:
            all_ref = [i.start() for i in re.finditer("References", article)]
        for index in all_ref: 
            reference_list.append(article[index:index+8000])

    return reference_list


In [56]:
#saving the references
ref_list2012 = find_reference(2012, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2012_full_txt.txt")
ref_list2021 = find_reference(2021, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2021_full_txt.txt")

In [57]:
def create_ref_dic(ref_list):
    year={}
    i=0
    for element in ref_list:
        year[i] = []
        references = [i.start() for i in re.finditer("\([0-9][0-9][0-9][0-9]\)", element)]
        for index in references:
            num = element[index+1:index+5]
            if int(num) < 2023: #eliminating some mistakes with numbers that are clearly not years
                year.get(i).append(num)
        i+=1
    return year


In [94]:
def add_references(df, ref_list_all):
    
    
    row_list = df[df['category'] == 'category: classification'].index
    df_class = df[df['category'] == 'category: classification']
    
    ref_dic = create_ref_dic(ref_list_all)
    
    oldest_list = []
    newest_list = []
    range_list = []
    class_dic = {}
    num_ref = []

    for element in row_list:
        ref_dic.get(element).sort()
        class_dic[element] = ref_dic.get(element) #need this otherwise the sort is forgotten
        ref_list = class_dic.get(element)
        oldest_list.append(int(ref_list[0]))
        newest_list.append(int(ref_list[len(ref_list)-1]))
        range_list.append(int(ref_list[len(ref_list)-1])- int(ref_list[0]))
        num_ref.append(len(ref_list))
   
    
    data = {'index': row_list,
            'Number of references': num_ref,
            'Oldest reference': oldest_list,
            'Newest reference' : newest_list,
            'Range of references': range_list
    } 
    
    ref = pd.DataFrame(data)
    
    df_class = df_class.reset_index()
    
    df_class = df_class.merge(ref, on = 'index')
 
    
    return df_class 

    

In [95]:
df_2012class = add_references(df_2012, ref_list2012)

df_2021class = add_references(df_2021, ref_list2021)

In [98]:
#saving updated database to csv
df_2012class.to_csv("database_miccai_2012_with_ref.csv")
df_2021class.to_csv("database_miccai_2021_with_ref.csv")

In [97]:
df_2021class

,index,Title,Authors,Page numbers,DOI,Year of publication,Part of publication,category,Number of references,Oldest reference,Newest reference,Range of references
0,8,Consistent Segmentation of Longitudinal Brain ...,"Jie Wei, Feng Shi, Zhiming Cui, Yongsheng Pan,...",89-98,/chapter/10.1007/978-3-030-87193-2_9,2021,1,category: classification,16,1999,2021,22
1,17,Improved Brain Lesion Segmentation with Anatom...,"Chenghao Liu, Xiangzhu Zeng, Kongming Liang, Y...",186-195,/chapter/10.1007/978-3-030-87193-2_18,2021,1,category: classification,17,1986,2021,35
2,18,CarveMix: A Simple Data Augmentation Method fo...,"Xinru Zhang, Chenghao Liu, Ni Ou, Xiangzhu Zen...",196-205,/chapter/10.1007/978-3-030-87193-2_19,2021,1,category: classification,16,2000,2021,21
3,25,Partially-Supervised Learning for Vessel Segme...,"Yanyu Xu, Xinxing Xu, Lei Jin, Shenghua Gao, R...",271-281,/chapter/10.1007/978-3-030-87193-2_26,2021,1,category: classification,26,1989,2020,31
4,30,Multi-compound Transformer for Accurate Biomed...,"Yuanfeng Ji, Ruimao Zhang, Huijie Wang, Zhen L...",326-336,/chapter/10.1007/978-3-030-87193-2_31,2021,1,category: classification,25,2012,2021,9
...,...,...,...,...,...,...,...,...,...,...,...,...
289,522,Adversarial Learning of Cancer Tissue Represen...,"Adalberto Claudio Quiros, Nicolas Coudray, Ann...",602-612,/chapter/10.1007/978-3-030-87237-3_58,2021,8,category: classification,18,2011,2021,10
290,524,USCL: Pretraining Deep Ultrasound Image Diagno...,"Yixiong Chen, Chunhui Zhang, Li Liu, Cheng Fen...",627-637,/chapter/10.1007/978-3-030-87237-3_60,2021,8,category: classification,25,2003,2021,18
291,525,Identifying Quantitative and Explanatory Tumor...,"Peng Wan, Chunrui Liu, Fang Chen, Jing Qin, Da...",638-647,/chapter/10.1007/978-3-030-87237-3_61,2021,8,category: classification,13,2003,2017,14
292,526,Weakly-Supervised Ultrasound Video Segmentatio...,"Ruiheng Chang, Dong Wang, Haiyan Guo, Jia Ding...",648-658,/chapter/10.1007/978-3-030-87237-3_62,2021,8,category: classification,28,2008,2020,12
